In [2]:
import pandas as pd
import pandas_gbq
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

In [ ]:
query = """with initial_dataset as (
  select
    event_name,
    optional_label,
    event_timestamp,
    event_date,
    user_id,
    user_pseudo_id,
    ga_session_id,
    user_history_event_number,
    ga_session_number,
    session_event_number,
    platform,
    device_language,
    country
  from
    analytics_151430920.firebase_facts_events as eve
  where
    eve.event_timestamp >= '2020-11-01'
  and
    eve.event_name in ('sv__OnBoardingTutorialView','sv__WelcomeCarouselDialog','sv__NewPremiumAct','TryToBuyNewPAct','TryToBuyNewPF','ClickOnWholeView','StartListening','sv__StoryDetails','StartNStory','StartBekids','FabClickedPremium','PremiumBarClickedSD','PremiumBarClickedMain','AAPageA','ProPageA','GoldPageA','AAPageF','ProPageF','GoldPageF','EnterFcMore','sv__GlossaryF','LibraryClicked','sv__Libraries','PlayPrevParagraphFromButton','PlayNextParagraphFromButton','in_app_purchase','PurchaseNormal')
),   

premium_promo_shown as (
  select distinct
    user_pseudo_id,
    first_value(event_timestamp) over (partition by user_pseudo_id order by event_timestamp) as first_premium_promo_shown_timestamp
  from
    initial_dataset
  where
    event_name = 'sv__WelcomeCarouselDialog'
),

purchases as (
  select distinct
    'in_app_purchase' as event_name,
    first_value(event_timestamp) over (partition by user_pseudo_id order by event_timestamp) as first_purchase_timestamp,
    first_value(event_date) over (partition by user_pseudo_id order by event_timestamp) as first_purchase_date,
    first_value(optional_label) over (partition by user_pseudo_id order by event_timestamp) as optional_label,
    last_value(user_id) over (partition by user_pseudo_id order by event_timestamp) as user_id,
    user_pseudo_id,
    first_value(ga_session_id) over (partition by user_pseudo_id order by event_timestamp) as ga_session_id,
    first_value(user_history_event_number) over (partition by user_pseudo_id order by event_timestamp) as user_history_event_number,
    first_value(session_event_number) over (partition by user_pseudo_id order by event_timestamp) as session_event_number,
    first_value(ga_session_number) over (partition by user_pseudo_id order by event_timestamp) as ga_session_number,
    first_value(platform) over (partition by user_pseudo_id order by event_timestamp) as platform,
    first_value(device_language) over (partition by user_pseudo_id order by event_timestamp) as device_language,
    first_value(country) over (partition by user_pseudo_id order by event_timestamp) as country
  from
    initial_dataset
  where
    event_name in ('in_app_purchase','PurchaseNormal')
), 

all_purchases as (
  select
    pu.event_name,
        case 
          when ps.first_premium_promo_shown_timestamp is null then 'iOS' 
          when pu.first_purchase_timestamp > timestamp_add(ps.first_premium_promo_shown_timestamp, interval 30 minute) then 'Android - no discount' else 'Android - discount' end as purchase_with_discount,
    ps.user_pseudo_id as premium_shown_user_pseudo_id,

    pu.first_purchase_timestamp,
    pu.first_purchase_date,
    ps.first_premium_promo_shown_timestamp,
    pu.optional_label,
    pu.user_id,
    pu.user_pseudo_id,
    pu.ga_session_id,
    pu.ga_session_number,
    pu.user_history_event_number,
    pu.session_event_number,
    pu.platform,
    pu.device_language,
    pu.country

  from
    purchases as pu
  left join
    premium_promo_shown as ps on ps.user_pseudo_id = pu.user_pseudo_id
)

select
  first_purchase_date,
  purchase_with_discount,
  case when ga_session_number > 1 then '2+' else '1' end as nth_session,
  count(*)
from
  all_purchases
group by 1,2,3
order by 1 desc, 2 desc, 3 desc;"""

df = pandas_gbq.read_gbq(query, project_id= 'beelinguapp') 

In [10]:
all_events_query = """with initial_dataset as (
  select
    event_name,
    optional_label,
    event_timestamp,
    event_date,
    user_id,
    user_pseudo_id,
    ga_session_id,
    user_history_event_number,
    session_event_number,
    platform,
    device_language,
    country
  from
    analytics_151430920.firebase_facts_events as eve
  where
    eve.event_timestamp >= '2020-11-01'
  and
    eve.event_name in ('sv__OnBoardingTutorialVie','sv__OnBoardingTutorialVi','sv__WelcomeCarouselDialog','sv__NewPremiumAct','TryToBuyNewPAct','TryToBuyNewPF','ClickOnWholeView','StartListening','sv__StoryDetails','StartNStory','StartBekids','FabClickedPremium','PremiumBarClickedSD','PremiumBarClickedMain','AAPageA','ProPageA','GoldPageA','AAPageF','ProPageF','GoldPageF','EnterFcMore','sv__GlossaryF','LibraryClicked','sv__Libraries','PlayPrevParagraphFromButton','PlayNextParagraphFromButton','in_app_purchase','PurchaseNormal')
),

onboarding as (
  select distinct
    'onboarding' as event_name,
    first_value(event_timestamp) over (partition by user_pseudo_id order by event_timestamp) as onboarding_timestamp,
    first_value(event_date) over (partition by user_pseudo_id order by event_timestamp) as onboarding_date,
    first_value(optional_label) over (partition by user_pseudo_id order by event_timestamp) as optional_label,
    last_value(user_id) over (partition by user_pseudo_id order by event_timestamp) as user_id,
    user_pseudo_id,
    first_value(ga_session_id) over (partition by user_pseudo_id order by event_timestamp) as ga_session_id,
    first_value(user_history_event_number) over (partition by user_pseudo_id order by event_timestamp) as user_history_event_number,
    first_value(session_event_number) over (partition by user_pseudo_id order by event_timestamp) as session_event_number,
    first_value(platform) over (partition by user_pseudo_id order by event_timestamp) as platform,
    first_value(device_language) over (partition by user_pseudo_id order by event_timestamp) as device_language,
    first_value(country) over (partition by user_pseudo_id order by event_timestamp) as country
  from
    initial_dataset
  where
    event_name in ('sv__OnBoardingTutorialVi','sv__OnBoardingTutorialVie')
),

purchases as (
  select distinct
    'in_app_purchase' as event_name,
    first_value(event_timestamp) over (partition by user_pseudo_id order by event_timestamp) as first_purchase_timestamp,
    first_value(event_date) over (partition by user_pseudo_id order by event_timestamp) as first_purchase_date,
    first_value(optional_label) over (partition by user_pseudo_id order by event_timestamp) as optional_label,
    last_value(user_id) over (partition by user_pseudo_id order by event_timestamp) as user_id,
    user_pseudo_id,
    first_value(ga_session_id) over (partition by user_pseudo_id order by event_timestamp) as ga_session_id,
    first_value(user_history_event_number) over (partition by user_pseudo_id order by event_timestamp) as user_history_event_number,
    first_value(session_event_number) over (partition by user_pseudo_id order by event_timestamp) as session_event_number,
    first_value(platform) over (partition by user_pseudo_id order by event_timestamp) as platform,
    first_value(device_language) over (partition by user_pseudo_id order by event_timestamp) as device_language,
    first_value(country) over (partition by user_pseudo_id order by event_timestamp) as country
  from
    initial_dataset
  where
    event_name in ('in_app_purchase','PurchaseNormal')
),

relevant_events as (
  select
    ini.event_name,
    ini.event_timestamp,
    ini.event_date,
    ini.optional_label,
    ini.user_id,
    ini.user_pseudo_id,
    ini.ga_session_id,
    ini.user_history_event_number,
    ini.session_event_number,
    ini.platform,
    ini.device_language,
    ini.country
  from
    initial_dataset as ini
  left join
    purchases as pur on pur.user_pseudo_id = ini.user_pseudo_id
  left join
    onboarding as onb on onb.user_pseudo_id = ini.user_pseudo_id
  where
    ini.event_name not in ('in_app_purchase','PurchaseNormal','sv__OnBoardingTutorialVi','sv__OnBoardingTutorialVie')
  and
    ini.event_timestamp between onb.onboarding_timestamp and timestamp_add(onb.onboarding_timestamp, interval 7 day)
  union all
  select
    rep.event_name,
    rep.first_purchase_timestamp as event_timestamp,
    rep.first_purchase_date as event_date,
    rep.optional_label,
    rep.user_id,
    rep.user_pseudo_id,
    rep.ga_session_id,
    rep.user_history_event_number,
    rep.session_event_number,
    rep.platform,
    rep.device_language,
    rep.country
  from
    purchases as rep
)

select
  user_pseudo_id,
  event_name,
  count(event_timestamp) as occurrences_count,
  min(event_timestamp) as first_event_timestamp,
  max(event_timestamp) as last_event_timestamp
from
  relevant_events
group by 1,2;
"""
df = pandas_gbq.read_gbq(all_events_query, project_id= 'beelinguapp')
df.head()


Downloading: 100%|██████████| 470337/470337 [00:28<00:00, 16551.57rows/s]


,user_pseudo_id,event_name,occurrences_count,first_event_timestamp,last_event_timestamp
0,94a2e7db90eaa2615b9d66192e28526c,in_app_purchase,1,2020-11-05 07:37:49.570000+00:00,2020-11-05 07:37:49.570000+00:00
1,3CFE7A2AAE17476CB6EEF20D996F1C8C,in_app_purchase,1,2020-12-01 08:14:30.664000+00:00,2020-12-01 08:14:30.664000+00:00
2,f47c9e812b8daf62970bf96bc3a8d4db,in_app_purchase,1,2020-12-04 02:19:40.779000+00:00,2020-12-04 02:19:40.779000+00:00
3,1600677bf3c4ec9719e0f6104d776f18,in_app_purchase,1,2020-12-03 16:56:40.859000+00:00,2020-12-03 16:56:40.859000+00:00
4,2398d78a9f5072e22968408b50c22334,in_app_purchase,1,2020-11-13 17:44:00.894000+00:00,2020-11-13 17:44:00.894000+00:00
